In [ ]:
import glob, os
import matplotlib.pyplot as plt
from skimage import io
import cv2
import shutil
import numpy as np
import SimpleITK as sitk
import read_roi
from read_roi import read_roi_file
from collections import OrderedDict
import pandas as pd
import shutil
import preprocessing_tool as procTool


def IoU(box1, box2):
    # box = (x1, y1, x2, y2)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    return iou


In [ ]:
######## Final
#####0721
sort_mode = True
date='0713'
reSize= 256
cropSize = 128
reSpacing = 0.695
partial_region = 64
t_option = '098_Random'
model = 'resnet101'
d_option = 'trainfold0_conbine_Crop{}to{}_wind_Zcnstr_intv5_partial{}_{}'.format(str(cropSize),str(reSize),partial_region, date)
h_parameter = 'f0p2_lr0001'
initListPath = './DVT_Det_retinanet_tens14/keras_retinanet/bin/evaluation_log/'


############sorting result by image name
if sort_mode==True:
# testList = pd.read_csv('./test_Data/FP_check.csv')
    initList = pd.read_csv(initListPath+'{}_{}_{}_{}.csv'.format(t_option, model, d_option, h_parameter))

    
    initList.sort_values(by=['img_name'], axis=0, inplace=True)
    initList.to_csv('../data/Result_figure/sorted_evaluation/{}_{}_{}_{}.csv'.format(t_option, model, d_option, h_parameter),index=False)
    # print(testList)

    # init_save_path = '../data/Result_figure/postprocessed_result/{}_{}_{}_{}'.format(t_option, model, d_option, h_parameter)
    # os.makedirs(init_save_path, exist_ok=True)

    testList = pd.read_csv('../data/Result_figure/sorted_evaluation/{}_{}_{}_{}.csv'.format(t_option, model, d_option, h_parameter))
else:
    # init_save_path = '../data/Result_figure/postprocessed_result/{}_{}_{}_{}'.format(t_option, model, d_option, h_parameter)
    testList = pd.read_csv(initListPath+'{}_{}_{}_{}.csv'.format(t_option, model, d_option, h_parameter))

# os.makedirs(init_save_path, exist_ok=True)

# testList = pd.read_csv('../data/Result_figure/sorted_evaluation/test_list.csv')

init_save_path = '../data/Result_figure/postprocessed_result/{}_{}_{}_{}'.format(t_option, model, d_option, h_parameter)
os.makedirs('./{}/text/'.format(init_save_path),exist_ok = True)

past_img = testList['img_name'][0]

init_results = pd.DataFrame(columns=['img_path','img_name','img_label',
                                       'GT_x1','GT_y1','GT_x2','GT_y2',
                                       'Pred_x1','Pred_y1','Pred_x2','Pred_y2',
                                       'probability','IoU','status'])

loc_array = np.zeros((1, 4), dtype = np.float32)


# os.makedirs('./{}/image/'.format(init_save_path))
loc_num=0

for tl in range(len(testList)):    ###line by line
    print('\n')
    print('#'*50)
    print(tl)
    one_img_name = testList['img_name'][tl]


    if tl ==0 : ##initial image
        predict_loc = testList['Pred_x1'][tl],testList['Pred_y1'][tl], testList['Pred_x2'][tl], testList['Pred_y2'][tl]
        predict_loc = np.array(predict_loc).astype(float).astype(int)
        loc_array[0] = predict_loc
        past_img = one_img_name
        print('start')
        loc_num+=1


    else: 
        if tl == len(testList)-1:   ###last image
            predict_loc = testList['Pred_x1'][tl],testList['Pred_y1'][tl], testList['Pred_x2'][tl], testList['Pred_y2'][tl]
            predict_loc = np.expand_dims(np.array(predict_loc).astype(float).astype(int), axis=0)
            loc_array = np.concatenate((loc_array,predict_loc),axis=0)
            loc_num=loc_array.shape[0]

            same_img=False
        else:
            if past_img == one_img_name:
                same_img = True
            else:
                same_img = False

        print('past: {} / present: {}'.format(past_img, one_img_name))

        if testList['status'][tl]=='FN':
            change_results = pd.concat([init_results, testList.iloc[[tl],:]], axis=0)
            init_results = change_results
            # init_results.append(testList.iloc[[tl],:])
            continue

        if same_img == True:
            loc_num+=1

            print('same image ++, image number:{}'.format(tl))

            predict_loc = testList['Pred_x1'][tl],testList['Pred_y1'][tl], testList['Pred_x2'][tl], testList['Pred_y2'][tl]
            predict_loc = np.expand_dims(np.array(predict_loc).astype(float).astype(int), axis=0)
            loc_array = np.concatenate((loc_array,predict_loc),axis=0)

            
        
        else:
            print('before {} image. start to next images {}'.format(loc_num, tl))
            print('check array number: {}, {}'.format(loc_array.shape, loc_num))


            if loc_num!=1:
                one_img_prob = []     
                init_local = tl - loc_num
                one_img_check = testList.iloc[[init_local],:]
                
                one_img_prob.append(testList['probability'][init_local])
                each_index = []
                no_overlap = []
                print('loc_num = {}'.format(loc_num))

                for k in range(loc_num-1):
                    a = tl - loc_num + k
                    one_img_check = pd.concat([one_img_check, testList.iloc[a,:]], axis=0)
                    each_index.append(a)
                    each_prob1 = testList['probability'][a]
                    print('present image: {}'.format(a))

                    for j in range(1,loc_num-k):
                        print('k, j, loc_num-1: {},{},{}'.format(k, j, loc_num-1))
                        print('a & a+j: ', a, a+j)
                        print('compare {}'.format(one_img_name))
                        print('compare {} & {}'.format(testList['probability'][a], testList['probability'][a+j]))
                        print('location {} and {}'.format(loc_array[k], loc_array[k+j]))

                        each_iou = IoU(loc_array[k], loc_array[k+j])
                        print('IoU: {}'.format(each_iou))

                        if each_iou<0.8 :

                            no_overlap.append(testList['probability'][a])
                            no_overlap.append(testList['probability'][a+j])
                            print('no_overlap {} & {}'.format(testList['probability'][a], testList['probability'][a+j]))

                        else:
                            if  testList['probability'][a] > testList['probability'][a+j]:
                                change_results = pd.concat([init_results, testList.iloc[[a],:]], axis=0)
                                init_results = change_results
                                print('higher probability : {}'.format(testList['probability'][a]))
                            else:

                                change_results = pd.concat([init_results, testList.iloc[[a+j],:]], axis=0)
                                init_results = change_results
                                print('higher probability : {}'.format(testList['probability'][a+j]))

                count={}
                print('no_overlap:', no_overlap)

                for i in no_overlap:

                    try: count[i] += 1
                    except: count[i] = 1

                print('count:',count)
                lists_key = list(count.keys())
                lists_value = list(count.values())
                print("no overlap list's key and value: \n",lists_key, lists_value)

                ###############################################################################
                for ck in range(len(count.keys())):
                    if loc_num>2:
                        if lists_value[ck] == loc_num-1:
                            change_results = pd.concat([init_results, one_img_check[one_img_check['probability']==lists_key[ck]]], axis=0)  ####다른 이미지에서 같은 probability가 있을 수도 있는 경우 배제
                            init_results = change_results
                            print('#'*20)
                            print('no overlap append: {} '.format(lists_value[ck]))
                    elif loc_num==2:
                        if len(no_overlap)!=2:
                            change_results = pd.concat([init_results, one_img_check[one_img_check['probability']==lists_key[ck]]], axis=0)  
                            init_results = change_results
                        else:
                            print('check loc_num and no overlap')

                    else:
                        print('wrong loc_num')


            else:
                print('{} has one result'.format(one_img_name))
                change_results = pd.concat([init_results, testList.iloc[[tl-1],:]], axis=0)
                init_results = change_results

                past_img = one_img_name

            loc_num=1

            past_img = one_img_name
            loc_array = np.zeros((1, 4), dtype = np.float32)
            predict_loc = testList['Pred_x1'][tl],testList['Pred_y1'][tl], testList['Pred_x2'][tl], testList['Pred_y2'][tl]
            predict_loc = np.expand_dims(np.array(predict_loc).astype(float).astype(int), axis=0)
            loc_array[0] = predict_loc

    init_results= init_results.drop_duplicates()    ####removing last
    print('done {}'.format(tl))
init_results.to_csv('./{}/text/reductionFP_{}.csv'.format(init_save_path,date))